In [300]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.model_selection import KFold
import time

pd.set_option('display.max_columns', 50)

from evaluation_and_testing.testing import evaluate_train_test_split_explicit
from evaluation_and_testing.testing import evaluate_leave_one_out_explicit
from evaluation_and_testing.testing import evaluate_train_test_split_implicit
from evaluation_and_testing.testing import evaluate_leave_one_out_implicit

from recommenders.recommender import Recommender

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [2]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')

display(ml_movies_df.head(10))

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of left interactions: {}".format(len(ml_ratings_df)))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Number of left interactions: 9692


# Baseline recommenders

**Task 1.** Implement the MostPopularRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [363]:




class MostPopularRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.most_popular_items = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        # Write your code here
        items_popularity = defaultdict(lambda: 0)

        p = interactions_df.loc[:,['item_id', 'user_id']].groupby(by='item_id').count()
        p = p.sort_values(by='user_id', ascending=False)
        
        
        self.most_popular_items = p.index[:100]
        
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': self.most_popular_items[:n_recommendations],
                                                 'score': [1 / (i + 1) for i in range(n_recommendations)]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations


    



# Quick test of the recommender


most_popular_recommender = MostPopularRecommender()
most_popular_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = most_popular_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

most_popular_recommender = MostPopularRecommender()

most_popular_results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    most_popular_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

most_popular_results = pd.DataFrame(
    most_popular_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
display(HTML(most_popular_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,50,1.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
1,1,780,0.5,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
2,1,858,0.333333,"Godfather, The (1972)",Crime|Drama
3,1,595,0.25,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
4,1,1704,0.2,Good Will Hunting (1997),Drama|Romance
5,1,1213,0.166667,Goodfellas (1990),Crime|Drama
6,1,2716,0.142857,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi
7,1,586,0.125,Home Alone (1990),Children|Comedy
8,1,208,0.111111,Waterworld (1995),Action|Adventure|Sci-Fi
9,1,7438,0.1,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749



**Task 2.** Implement the HighestRatedRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [369]:




class HighestRatedRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.highest_rated_items = None
        self.highest_ratings = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        # Write your code here
        ratings = interactions_df.loc[:, ['item_id', 'rating']].groupby('item_id').mean()
        ratings_count = interactions_df.loc[:, ['item_id', 'rating']].groupby(by='item_id').count()
        ratings = ratings.loc[ratings_count['rating'] >= 50]
        ratings = ratings.sort_values('rating', ascending=False)
        self.highest_rated_items = ratings.index[:100]
        self.highest_ratings = ratings['rating'][:100]
        
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            # Write your code here
            
            user_recommendations = pd.DataFrame({
                'user_id':  user['user_id'],
                'item_id': self.highest_rated_items[:n_recommendations],
                'score': self.highest_ratings[:n_recommendations] / 5
               
            })
            recommendations = pd.concat([recommendations, user_recommendations])
        return recommendations
    
# Quick test of the recommender

highest_rated_recommender = HighestRatedRecommender()
highest_rated_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = highest_rated_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id')
print("Recommendations")
display(HTML(recommendations.to_html()))

highest_rated_recommender = HighestRatedRecommender()

highest_rated_results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
    highest_rated_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

highest_rated_results = pd.DataFrame(
    highest_rated_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
print(highest_rated_recommender.highest_rated_items)

display(HTML(highest_rated_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,126088,1.0,A Flintstones Christmas Carol (1994),Animation|Children|Comedy
1,2,126088,1.0,A Flintstones Christmas Carol (1994),Animation|Children|Comedy
2,6,126088,1.0,A Flintstones Christmas Carol (1994),Animation|Children|Comedy
3,1,1151,1.0,Lesson Faust (1994),Animation|Comedy|Drama|Fantasy
4,2,1151,1.0,Lesson Faust (1994),Animation|Comedy|Drama|Fantasy
5,6,1151,1.0,Lesson Faust (1994),Animation|Comedy|Drama|Fantasy
6,1,5468,1.0,20 Million Miles to Earth (1957),Sci-Fi
7,2,5468,1.0,20 Million Miles to Earth (1957),Sci-Fi
8,6,5468,1.0,20 Million Miles to Earth (1957),Sci-Fi
9,1,171749,1.0,Death Note: Desu nôto (2006–2007),(no genres listed)


         rating
item_id        
126088      5.0
1151        5.0
5468        5.0
171749      5.0
64501       5.0
...         ...
151745      0.5
173307      0.5
54768       0.5
144352      0.5
6557        0.5

[999 rows x 1 columns]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.0,0.0,0.0,0.003333,0.0,0.0,0.0,0.001052


**Task 3.** Implement the RandomRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [72]:
class RandomRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self, seed=0):
        """
        Initialize base recommender params and variables.
        
        :param int seed: Seed for the random number generator.
        """
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
        self.items = []
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        self.items = items_df['item_id'].unique().tolist()
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Write your code here
        for i, user in users_df.iterrows():
            recommendation = pd.DataFrame(
            {
                'user_id': user['user_id'],
                'item_id': self.rng.choice(self.items, n_recommendations, replace=False),
                'score': 1.0
            })
            recommendations = pd.concat([recommendations, recommendation])
            
        recommendations = recommendations.reset_index(drop=True)

        return recommendations
    
# Quick test of the recommender

random_recommender = RandomRecommender(seed=6789)
random_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = random_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = recommendations.merge(ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

random_recommender = RandomRecommender(seed=seed)

random_results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
    random_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=seed))]

random_results = pd.DataFrame(
    random_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(random_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,472,1.0,I'll Do Anything (1994),Comedy|Drama
1,1,26631,1.0,Alice (Neco z Alenky) (1988),Animation|Fantasy|Mystery
2,1,63276,1.0,Crows Zero (Kurôzu zero) (2007),Action
3,1,2815,1.0,Iron Eagle (1986),Action|War
4,1,31553,1.0,Double Dragon (1994),Action|Adventure|Sci-Fi
5,1,46335,1.0,"Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)",Action|Crime|Drama|Thriller
6,1,1107,1.0,Loser (1991),Comedy
7,1,42728,1.0,Tristan & Isolde (2006),Drama|Romance
8,1,4558,1.0,Twins (1988),Comedy
9,1,77866,1.0,Robin Hood (2010),Action|Adventure|Drama|Romance|War


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomRecommender,0.0,0.003333,0.01,0.01,0.0,0.002103,0.004682,0.004682


# Linear Regression Recommender

For every movie we transform its genres into one-hot encoded features and we normalize them, for every user we count percentages for all genres how often do they appear among films watched by the user, we multiply both vectors (for the item and the user) to obtain explanaytory variables, and then we fit a linear regression model to those features and actual ratings.

**Task 4.** Implement the calculate_user_genres method for calculating a DataFrame with one row per user and columns corresponding to genres (e.g. 'user_action', 'user_drama') with values calculated as follows:

- count the number of times a given user watched a given genre,
- apply a natural logarithm to this value plus one,
- normalize those values so that the sum of them over all columns within a row is equal to 1.

Implement the calculate_item_genres method for replacing the 'genres' column with one column per genre (e.g. 'action', 'drama') with values calculated as follows:

- place 1 in every column for which the genre appears in genres,
- normalize those values so that the sum of them over all genre columns within a row is equal to 1.

If item_features is None, then first find all genres and prepare a list of them. If item_features is not None, then create columns baed on this list. Return both the transformed DataFrame and the list of genres. Do not use MultiLabelBinarizer.

Note that in this second method you have to preserve the remaining structure of the DataFrame.

Evaluate the LinearRegressionRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [349]:
# Prepare data for tests
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")

In [350]:
def calculate_user_genres(interactions_df):
    users_df = interactions_df.copy()
    # Write your code here
    users_df = users_df.explode('genres')
    users_df['val'] = 1
    users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
    users_df = users_df.apply(lambda x: np.log(x+1) )
    users_df = users_df.rename_axis(None, axis=1).fillna(0)
    users_df = users_df.add_prefix('user_')
    users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
    return users_df



In [86]:
# Test the calculate_user_genres method
display(calculate_user_genres(interactions_df))

,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western,sum
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.102311,0.093749,0.059149,0.076449,0.093749,0.083027,0.000000,0.068670,0.059149,0.000000,0.000000,0.000000,0.029575,0.029575,0.0,0.076449,0.068670,0.083027,0.046875,0.029575,1.0
2,0.144386,0.000000,0.000000,0.000000,0.000000,0.144386,0.000000,0.255742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.091097,0.091097,0.182194,0.000000,0.091097,1.0
3,0.256885,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162077,0.000000,0.162077,0.000000,0.000000,0.000000,0.0,0.000000,0.256885,0.162077,0.000000,0.000000,1.0
4,0.035810,0.035810,0.056758,0.071621,0.128379,0.092569,0.000000,0.143242,0.056758,0.035810,0.000000,0.035810,0.071621,0.000000,0.0,0.107431,0.000000,0.056758,0.035810,0.035810,1.0
5,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.000000,0.133333,0.066667,0.000000,0.000000,0.066667,0.066667,0.066667,0.0,0.133333,0.000000,0.066667,0.066667,0.066667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.065798,0.060970,0.029090,0.047444,0.096320,0.073415,0.018354,0.111728,0.063494,0.018354,0.036708,0.029090,0.018354,0.047444,0.0,0.089163,0.055062,0.073415,0.047444,0.018354,1.0
607,0.113226,0.075484,0.000000,0.037742,0.105955,0.087634,0.000000,0.130566,0.037742,0.000000,0.075484,0.000000,0.037742,0.037742,0.0,0.059820,0.087634,0.075484,0.037742,0.000000,1.0
608,0.085929,0.067345,0.045723,0.070753,0.096567,0.075138,0.000000,0.083142,0.067345,0.000000,0.065454,0.028035,0.028035,0.041071,0.0,0.061191,0.075138,0.081099,0.028035,0.000000,1.0


In [379]:
from sklearn.preprocessing import MultiLabelBinarizer
def calculate_item_genres(item_genres, item_features=None):
# Implement the calculate_item_genres method for replacing the 'genres' column with one column per genre 
# (e.g. 'action', 'drama') with values calculated as follows:
# place 1 in every column for which the genre appears in genres,
# normalize those values so that the sum of them over all genre columns within a row is equal to 1.
    item_genres = item_genres.copy()
    # Write your code here
    
    if item_features is None:
        g_list = item_genres.loc[:, 'genres']
        temp = item_genres.explode('genres')
        temp = temp.sort_values('genres')
        temp = pd.unique(temp.loc[:,'genres'])
        zeros = list()
        def check(row):
            a = [0 for i in range(len(temp))]
            for item in row:
                for i in range(len(temp)):
                    if item == temp[i]:
                        a[i] = 1
            zeros.append(a)
        g_list.apply(lambda x: check(x))
        zeros = np.array(zeros)
        
            
        item_genres = item_genres.join(
            pd.DataFrame(zeros,
                         columns=temp,
                         index=item_genres.index))
        

        item_genres[temp] = item_genres[temp] / item_genres[temp].sum(axis=1).values.reshape(-1, 1)
        
       
        item_features = list(temp)
        
    else:
        item_genres_pivot = item_genres[['item_id', 'genres']].explode('genres')
        item_genres_pivot['val'] = 1
        item_genres_pivot = item_genres_pivot.pivot_table(index='item_id', columns='genres', values='val')
        item_genres_pivot = item_genres_pivot.rename_axis(None, axis=1).fillna(0)
        item_genres_pivot = item_genres_pivot.reindex(item_features, axis=1).fillna(0).reset_index()
        item_genres = item_genres.merge(item_genres_pivot, on='item_id')
        
        item_genres[item_features] = item_genres[item_features] \
            / item_genres[item_features].sum(axis=1).values.reshape(-1, 1)
    
    return item_genres, item_features
    
from sklearn.preprocessing import MultiLabelBinarizer

def calculate_item_genres2(item_genres, item_features=None):
    # Write your code here
    
    item_genres = item_genres.copy()
    
    if item_features is None:
        mlb = MultiLabelBinarizer()
        item_genres = item_genres.join(
            pd.DataFrame(mlb.fit_transform(item_genres.pop('genres')),
                         columns=mlb.classes_,
                         index=item_genres.index))

        item_genres[mlb.classes_] = item_genres[mlb.classes_] \
            / item_genres[mlb.classes_].sum(axis=1).values.reshape(-1, 1)
        
        item_features = list(mlb.classes_)
    else:
        item_genres_pivot = item_genres[['item_id', 'genres']].explode('genres')
        item_genres_pivot['val'] = 1
        item_genres_pivot = item_genres_pivot.pivot_table(index='item_id', columns='genres', values='val')
        item_genres_pivot = item_genres_pivot.rename_axis(None, axis=1).fillna(0)
        item_genres_pivot = item_genres_pivot.reindex(item_features, axis=1).fillna(0).reset_index()
        item_genres = item_genres.merge(item_genres_pivot, on='item_id')
        
        item_genres[item_features] = item_genres[item_features] \
            / item_genres[item_features].sum(axis=1).values.reshape(-1, 1)
    return item_genres, item_features

In [381]:
# Test the calculate_item_genres method

item_genres, item_features = calculate_item_genres(interactions_df)
# display(item_genres)
print(item_features)

item_genres, item_features = calculate_item_genres(interactions_df, ['comedy', 'crime', 'drama', 'horror', 'test_category'])
# display(item_genres)
print(item_features)



['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'imax', 'musical', 'mystery', 'no_genres_listed', 'romance', 'sci_fi', 'thriller', 'war', 'western']
['comedy', 'crime', 'drama', 'horror', 'test_category']
['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'imax', 'musical', 'mystery', 'no_genres_listed', 'romance', 'sci_fi', 'thriller', 'war', 'western']
['comedy', 'crime', 'drama', 'horror', 'test_category']


In [ ]:
from sklearn.linear_model import LinearRegression

class LinearRegressionRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.model = None
        self.users_dict = None
        self.user_features = None
        self.item_features = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Transform genres to a more code-friendly form
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
        
        # Prepare user features

        users_df = calculate_user_genres(interactions_df)
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
                
        # Prepare item features
        
        interactions_df, self.item_features = calculate_item_genres(interactions_df)

        # Prepare input data and fit the model
    
        interactions_df[self.user_features] = interactions_df[self.user_features] \
            * interactions_df[self.user_features].values
        
        x = interactions_df.loc[:, self.user_features].values
        y = interactions_df['rating'].values
    
        self.model = LinearRegression().fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Transform the item to be scored into proper features
        
        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df, _ = calculate_item_genres(items_df, self.item_features)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
#             display(user_df)
#             display(items_df)
            input_df = items_df.copy()
            input_df[self.item_features] = items_df[self.item_features] * user_df.values
#             display(input_df)
            scores = self.model.predict(input_df.loc[:, self.item_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
    def _transform_genres(self, df):
        """
        Transforms a string with genres into a list of cleaned genre names.
        
        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df

In [354]:
# Quick test of the recommender

lr_recommender = LinearRegressionRecommender()
lr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = lr_recommender.recommend(pd.DataFrame([[1], [2]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
display(recommendations)

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

,user_id,item_id,score,title,genres
0,1,66335,4.190204,Afro Samurai: Resurrection (2009),Animation
1,1,160848,4.190204,The Red Turtle (2016),Animation
2,1,32657,3.797962,"Man Who Planted Trees, The (Homme qui plantait...",Animation|Drama
3,1,95004,3.796203,Superman/Doomsday (2007),Action|Animation
4,1,147286,3.741605,The Adventures of Sherlock Holmes and Doctor W...,Crime|Mystery
5,1,146028,3.741605,The Adventures of Sherlock Holmes and Dr. Wats...,Crime|Mystery
6,1,6713,3.732956,Millennium Actress (Sennen joyû) (2001),Animation|Drama|Romance
7,1,158872,3.696238,Sausage Party (2016),Animation|Comedy
8,1,163925,3.696238,"Wings, Legs and Tails (1986)",Animation|Comedy
9,1,175431,3.696238,Bobik Visiting Barbos (1977),Animation|Comedy


### Train-test split test

In [ ]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_train_test_split_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

### Leave-one-out test

In [ ]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_leave_one_out_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

# TF-IDF Recommender
TF-IDF stands for term frequency–inverse document frequency. Typically Tf-IDF method is used to assign keywords (words describing the gist of a document) to documents in a corpus of documents.

In our case we will treat users as documents and genres as words.

Term-frequency is given by the following formula:
<center>
$$
    \text{tf}(g, u) = f_{g, u}
$$
</center>
where $f_{g, i}$ is the number of times genre $g$ appear for movies watched by user $u$.

Inverse document frequency is defined as follows:
<center>
$$
    \text{idf}(g) = \log \frac{N}{n_g}
$$
</center>
where $N$ is the number of users and $n_g$ is the number of users with $g$ in their genres list.

Finally, tf-idf is defined as follows:
<center>
$$
    \text{tfidf}(g, u) = \text{tf}(g, u) \cdot \text{idf}(g)
$$
</center>

In our case we will measure how often a given genre appears for movies watched by a given user vs how often it appears for all users. To obtain a movie score we will take the average of its genres' scores for this user.

**Task 5.** Implement the following method for calculating a TF-IDF scores in a form of a dict (use defaultdict):

`{(1, 'action'): 0.45306430692185395, (1, 'adventure'): 0.39370003643934415, (1, 'animation'): 0.20886242957049514, ...}`

without using TfidfVectorizer (you can use loops).

Evaluate the TFIDFRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [382]:
def calculate_tf_idf_scores(interactions):
    # Write your code here
    user_genres = interactions.loc[:, ['user_id', 'genres']].copy()
    user_genres = user_genres.groupby('user_id').aggregate(lambda x: "|".join(x))
    user_genres.loc[:, 'genres'] = user_genres['genres'].str.replace("|", " ", regex=False)
    user_ids = user_genres.index.tolist()
    genres = list(set("|".join(interactions['genres'].tolist()).split("|")))
    genres_corpus = user_genres['genres'].tolist()
    genres_to_idx = {genre: idx for idx, genre in enumerate(genres)}

    
    fgi = np.zeros((len(user_ids),len(genres)))
    for i, u in enumerate(genres_corpus):
        for g in u.split(" "):
            fgi[i, genres_to_idx[g]] +=1
            
    idf = np.zeros(len(genres))
    for g in genres:
        idf[genres_to_idx[g]] = np.log(len(user_ids)) / np.sum(user_genres['genres'].str.contains(g, regex=False))
            
    tf_idf = fgi * idf
    
    tfidf_scores = defaultdict(lambda: 0.0)
    for u in range(tf_idf.shape[0]):
        for g in range(tf_idf.shape[1]):
            tfidf_scores[(user_ids[u], genres[g])] = tf_idf[u, g]
    
    return tfidf_scores



In [383]:
# Test the method
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
        
tfidf_scores = calculate_tf_idf_scores(interactions_df)
display(tfidf_scores)

defaultdict(<function __main__.calculate_tf_idf_scores.<locals>.<lambda>()>,
            {(1, 'sci_fi'): 1.2729788914477906,
             (1, 'thriller'): 1.7060590271175973,
             (1, 'romance'): 1.5340801351916244,
             (1, 'war'): 2.121743921370525,
             (1, 'mystery'): 0.6473875634104785,
             (1, 'crime'): 2.3086654110482208,
             (1, 'drama'): 0.6840579435170525,
             (1, 'animation'): 2.628413572457773,
             (1, 'documentary'): 0.0,
             (1, 'comedy'): 1.791840171673406,
             (1, 'children'): 3.1107481541030797,
             (1, 'musical'): 0.9297589489250216,
             (1, 'imax'): 0.0,
             (1, 'action'): 2.198046500486951,
             (1, 'adventure'): 2.6013213256521714,
             (1, 'fantasy'): 1.6501390107578158,
             (1, 'horror'): 0.0,
             (1, '(no_genres_listed)'): 0.0,
             (1, 'western'): 1.1786549963416462,
             (1, 'film_noir'): 0.0,
             (

In [271]:
class TFIDFRecommender(Recommender):
    """
    Recommender based on the TF-IDF method.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.tfidf_scores = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        self.tfidf_scores = defaultdict(lambda: 0.0)

        # Prepare the corpus for tfidf calculation
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
                
        self.tfidf_scores = calculate_tf_idf_scores(interactions_df)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Transform genres to a unified form used by the vectorizer
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
                
        # Score items    
        
        for uix, user in users_df.iterrows():
            items = []
            for iix, item in items_df.iterrows():
                score = 0.0
                for genre in item['genres']:
                    score += self.tfidf_scores[(user['user_id'], genre)]
                score /= len(item['genres'])
                items.append((item['item_id'], score))
                
            items = sorted(items, key=lambda x: x[1], reverse=True)
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [item[0] for item in items][:n_recommendations],
                                                 'score': [item[1] for item in items][:n_recommendations]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [272]:
# Print movies watched by user 1, 2

active_user_movies = ml_df.loc[(ml_df['user_id'] == 1) | (ml_df['user_id'] == 2)]
print("Active users history")
display(active_user_movies)

Active users history


,user_id,item_id,rating,timestamp,title,genres
572,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
2946,1,423,3.0,964982363,Blown Away (1994),Action|Thriller
3790,1,543,4.0,964981179,So I Married an Axe Murderer (1993),Comedy|Romance|Thriller
5338,1,780,3.0,964984086,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
5709,1,940,5.0,964982176,"Adventures of Robin Hood, The (1938)",Action|Adventure|Romance
7753,1,1213,5.0,964982951,Goodfellas (1990),Crime|Drama
9398,1,1500,4.0,964980985,Grosse Pointe Blank (1997),Comedy|Crime|Romance
10294,1,1777,4.0,964981230,"Wedding Singer, The (1998)",Comedy|Romance
10520,1,2000,4.0,964982211,Lethal Weapon (1987),Action|Comedy|Crime|Drama
10974,1,2046,4.0,964982563,Flight of the Navigator (1986),Adventure|Children|Sci-Fi


In [273]:
# Quick test of the recommender

tfidf_recommender = TFIDFRecommender()
tfidf_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = tfidf_recommender.recommend(pd.DataFrame([[1], [2]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(recommendations)

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
Recommendations


,user_id,item_id,score,title,genres
0,1,343,3.110748,"Baby-Sitters Club, The (1995)",Children
1,1,183301,3.110748,The Tale of the Bunny Picnic (1986),Children
2,1,313,2.869581,"Swan Princess, The (1994)",Animation|Children
3,1,3213,2.869581,Batman: Mask of the Phantasm (1993),Animation|Children
4,1,3799,2.869581,Pokémon the Movie 2000 (2000),Animation|Children
5,1,42191,2.869581,Luxo Jr. (1986),Animation|Children
6,1,85316,2.869581,Winnie the Pooh and Tigger Too (1974),Animation|Children
7,1,828,2.856035,"Adventures of Pinocchio, The (1996)",Adventure|Children
8,1,3345,2.856035,"Charlie, the Lonesome Cougar (1967)",Adventure|Children
9,1,5504,2.856035,Spy Kids 2: The Island of Lost Dreams (2002),Adventure|Children


### Train-test split test

In [274]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_train_test_split_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

comedy romance thriller action adventure sci_fi thriller crime drama adventure animation children fantasy comedy crime crime mystery thriller action adventure thriller action crime action adventure thriller adventure fantasy romance comedy crime romance animation children comedy musical adventure children sci_fi action adventure romance adventure animation children sci_fi action children fantasy drama war action drama war action thriller


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,TFIDFRecommender,0.013423,0.029083,0.042506,0.082774,0.013423,0.022717,0.028204,0.041327


Total evaluation time: 23.527312755584717


### Leave-one-out test

In [275]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

...

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy western adventure animation children sci_fi
crime mystery thriller action thriller comedy romance thriller action adventure sci_fi thriller action adventure romance crime drama comedy crime romance comedy romance action comedy crime drama adventure children sci_fi animation children comedy musical adventure animation children fantasy comedy crime action crime action comedy sci_fi action adventure thriller drama war action drama war action children fantasy adventure fantasy romance action adventure thriller comedy wester

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,TFIDFRecommender,0.003333,0.003333,0.01,0.02,0.003333,0.003333,0.005912,0.009039


Total evaluation time: 30.28802752494812


**Task 6\*.** Implement an SVRRecommender by replacing the LinearRegression model with the SVR model (`from sklearn.svm import SVR`).

Tune the C and epsilon params of the SVR model to obtain as good results as you can. 

To do tuning properly:

- divide the set into training, validation and test sets (randomly divide the dataset in proportions 60%-20%-20%),
- train the model with different sets of tunable parameters on the training set, 
- choose the best tunable params based on results on the validation set, 
- provide the final evaluation metrics on the test set for the best model obtained during tuning.

Recommended method of tuning: use hyperopt. Install the package using the following command: `pip install hyperopt`
    
Print the RMSE, MAPE and TRE on the test set. Use seed 6789.

In [276]:
from sklearn.svm import SVR
from sklearn.preprocessing import MultiLabelBinarizer

class SVRRecommender(Recommender):
    """
    SVR recommender class.
    """
    
    def __init__(self, kernel='rbf', C=1.0, epsilon=0.1):
        """
        Initialize base recommender params and variables.
        """
        self.model = None
        self.kernel = kernel
        self.C = C
        self.epsilon = epsilon
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Write your code here
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
#         print(interactions_df.head())
        
        x = interactions_df.loc[:, self.mlb.classes_].values
        y = interactions_df['rating'].values
    
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon).fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Write your code here
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            score = self.model.predict(items_df.loc[:, self.mlb.classes_].values)[0]
                
            user_recommendations = pd.DataFrame({'user_id': [user['user_id']],
                                                 'item_id': items_df.iloc[0]['item_id'],
                                                 'score': score})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
    
# Quick test of the recommender

svr_recommender = SVRRecommender()
svr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = svr_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(recommendations)

Recommendations


,user_id,item_id,score,title,genres
0,1,4,3.900184,Waiting to Exhale (1995),Comedy|Drama|Romance
1,2,4,3.900184,Waiting to Exhale (1995),Comedy|Drama|Romance
2,6,4,3.900184,Waiting to Exhale (1995),Comedy|Drama|Romance


### Manually test different param configurations

In [277]:
svr_recommender = SVRRecommender(kernel='rbf', C=1.0, epsilon=0.1)

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])



display(results)

,Recommender,RMSE,MRE,TRE
0,SVRRecommender,0.975211,0.339687,0.215497


### Tune the SVRRecommender

In [293]:
from hyperopt import hp, fmin, tpe, Trials

# Split into train_validation and test sets

shuffle = np.arange(len(ml_ratings_df))
rng.shuffle(shuffle)
shuffle = list(shuffle)

train_test_split = 0.8
split_index = int(len(ml_ratings_df) * train_test_split)

train_validation = ml_ratings_df.iloc[shuffle[:split_index]].loc[:, ['user_id', 'item_id', 'rating']]
test = ml_ratings_df.iloc[shuffle[split_index:]].loc[:, ['user_id', 'item_id', 'rating']]

# Tune

def loss(tuned_params):
    svr_recommender = SVRRecommender(kernel='rbf', C=tuned_params['C'], epsilon=tuned_params['epsilon'])
    rmse, mre, tre = evaluate_train_test_split_explicit(
        svr_recommender, train_validation, ml_movies_df, seed=seed)
#     return np.array([rmse, mre, tre])
    return rmse

########################################################################
# Write your code here 
# Read the code above and below and make sure you understand it
# Minimize the above loss function using hyperopt
# Save your best params under best_param_set
########################################################################
from hyperopt import hp, fmin, tpe
best_param_set = dict()
best_param_set['C'] = 1.0
best_param_set['epsilon'] = 1.0

space = hp.uniform('C', -100, 100)

best = fmin(fn=loss, space=space, algo=tpe.suggest, max_evals=1000)
# Best params

print("C = {}".format(best_param_set['C']))
print("epsilon = {}".format(best_param_set['epsilon']))
    
# Test

svr_recommender = SVRRecommender(C=best_param_set['C'], epsilon=best_param_set['epsilon'])

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, {'train': train_validation, 'test': test}, ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

  0%|                                                                                                                                                                                            | 0/1000 [00:00<?, ?trial/s, best loss=?]

job exception: 'float' object is not subscriptable



  0%|                                                                                                                                                                                            | 0/1000 [00:00<?, ?trial/s, best loss=?]


TypeError: 'float' object is not subscriptable

**Task 7.** Gather the results for LinearRegressionRecommender, SVRRecommender, TFIDFRecommender, MostPopularRecommender, HighestRatedRecommender, RandomRecommender from the evaluate_train_test_split_implicit method and print them as a single DataFrame.

In [ ]:
# Write your code here
linear = LinearRegressionRecommender()
svr = SVRRecommender()
tfidf = TFIDFRecommender()
most_popular = MostPopularRecommender()
highest_rated = HighestRatedRecommender()
random = RandomRecommender()

recommenders = [linear_recommender, svr_recommender, tfidf_recommender, most_popular_recommender, 
                highest_rated_recommender, random_recommender]

all_results = list()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_train_test_split_implicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(results)

all_results = pd.concat(all_results).reset_index(drop=True)
print(all_results)

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

...

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']

['user_action',
 'user_adventure',
 'user_animation',
 'user_children',
 'user_comedy',
 'user_crime',
 'user_documentary',
 'user_drama',
 'user_fantasy',
 'user_film_noir',
 'user_horror',
 'user_imax',
 'user_musical',
 'user_mystery',
 'user_no_genres_listed',
 'user_romance',
 'user_sci_fi',
 'user_thriller',
 'user_war',
 'user_western']